In [1]:
import pandas as pd
from ibmpairs import paw, authentication
import json
import numpy
from datetime import datetime, timedelta
import logging
import matplotlib.pyplot as plt
import re
import numpy as np
import traceback
import rasterio
import os
import zipfile
import traceback
import zipfile
import glob
from datetime import date

In [2]:
logging.basicConfig(level=logging.INFO)
pawLogger = logging.getLogger('ibmpairs.paw')
pawLogger.setLevel(logging.INFO)

In [3]:
with open("ibmpairskey.txt", "r") as f:
    pairs_key = f.read()

In [4]:
def authenticate(pairs_key):
    
    pairs_credentials = authentication.OAuth2(api_key=pairs_key)
    auth_header = {'Authorization': f'Bearer {pairs_credentials.jwt_token}'}
    PAIRS_SERVER   = 'https://pairs.res.ibm.com'
    PAIRS_CREDENTIALS = authentication.OAuth2(api_key=pairs_key)
    
    return PAIRS_SERVER, PAIRS_CREDENTIALS

In [5]:
PAIRS_SERVER, PAIRS_CREDENTIALS = authenticate(pairs_key)

In [6]:
# Define date and space parameters
date_start, date_end = None, None #Needed for defining the query for ERA5
bbox = None
iso8601 = '%Y-%m-%dT%H:%M:%SZ'

date_red = datetime(2016,8,3)

In [7]:
date_red.strftime(iso8601)

'2016-08-03T00:00:00Z'

In [8]:
areas_test_max = {
    'bbox_1' : ["47.2", "7", "47.3", "7.1" ]
    #'bbox_2' : ["47.20599","7.94597","47.46599","8.20597"]
}

In [9]:
#Relative Humidity: 100*(EXP((17.625*TD)/(243.04+TD))/EXP((17.625*T)/(243.04+T)))

query_vpd_json = {
    "layers" : [
        {     
            #Kelvin
            "type" : "raster", "id" : "49423",
            "aggregation" : "Mean",
            "alias": "temperature",
            "output" : "false"
        },
        {            
            #Kelvin
            "type" : "raster", "id" : "49422",
            "aggregation" : "Mean",
            "alias": "dewpoint",
            "output" : "false"
        },
        {
            #convert kelvin to rankine
            "alias": "temp_rankine",
            "expression": "$temperature * 1.8"
            
        },
        {
            #convert dewpoint kelvin to celcius
            "alias": "dewpoint_celcius",
            "expression": "$dewpoint - 273.15"
            
        },
        {
            #calculate saturation
            "alias": "saturation",
            "expression": " math:exp(( (math:pow(10,4) * -1.0440397) / $temp_rankine) + -11.29465 + (math:pow(10,-2) * -2.7022355 * $temp_rankine) + (math:pow(10,-5) * 1.289036 * math:pow($temp_rankine,2)) + (math:pow(10,-9) * -2.4780681 * math:pow($temp_rankine,3)) + (6.5459673 * math:log($temp_rankine))) "
        },
        {
            #convert kelvin to celcius
            "alias": "temp_celcius",
            "expression": "$temperature - 273.15"
            
        },
        {
            #calculate relative humidty
            "alias": "relative_humidity",
            "expression": "100 * (math:exp((17.625 * $dewpoint_celcius) / (243.04 + $dewpoint_celcius)) / (math:exp((17.625 * $temp_celcius) /(243.04 + $temp_celcius))))"
            
        },
        {
            #calculate partial presurre
            "alias": "partial_pressure",
            "expression": "$saturation * ($relative_humidity / 100) "
            
        },
        {
            #calculate vapour pressure deficit
            "alias": "vpd",
            "expression": "$saturation * (1-($relative_humidity /100))" 
            #or, vpd = $saturation - $partial_pressure
            
        },
               
    ],
    "name": "math_sample",
    "spatial" : {"type" : "square",  "coordinates" : bbox }, 
    "temporal" : {"intervals" : [
      {"start" : date_start, "end" : date_end}
  ]}
}

In [10]:
# Function that sets the date range
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [11]:
from datetime import date
# Define start and end date
start_date = date(2019, 10, 5)
end_date = date(2019, 10, 7)

iso8601 = '%Y-%m-%dT%H:%M:%SZ'

In [12]:
# 1st loop: Loop through regions (Currently it is just one region)
# 2nd loop: For the current region, loop through days

for bbox, coord in areas_test_max.items():
    for single_date in daterange(start_date, end_date):
        
        print(f'Trying for date {single_date}')
        
        #Authenticate pairs before every query
        PAIRS_SERVER, PAIRS_CREDENTIALS = authenticate(pairs_key)
        
        #Set the date range
        date_start = single_date.strftime(iso8601)
        date_end = (single_date + timedelta(1)).strftime(iso8601)

        #Fill the placeholders in the query
        query_vpd_json["temporal"]["intervals"][0]["start"] = date_start
        query_vpd_json["temporal"]["intervals"][0]["end"] = date_end
        query_vpd_json["spatial"]["coordinates"] = coord
        
        #Turn query to an actual json
        #query_json = json.dumps(queryEraJson_3)
        
        #Submit query
        query_vpd = paw.PAIRSQuery(query_vpd_json, PAIRS_SERVER, PAIRS_CREDENTIALS, authType='api-key', overwriteExisting=False)
        query_vpd.submit()

        query_vpd.poll_till_finished()
        query_vpd.download()
        query_vpd.create_layers()

Trying for date 2019-10-05


INFO:ibmpairs.paw:PAIRS query JSON initialized.
INFO:ibmpairs.paw:Download directory './downloads' created.
INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1666324800_39115011
INFO:ibmpairs.paw:Here we go, PAIRS query result successfully downloaded as '1666324800_39115011_25ed55dc01e9e34073ddf8f81296f62d.zip'.
INFO:ibmpairs.paw:Data acknowledgement successfully loaded, print with `self.print_data_acknowledgement()`
INFO:ibmpairs.paw:PAIRS meta data loaded from 'output.info'.


Trying for date 2019-10-06


INFO:ibmpairs.paw:PAIRS query JSON initialized.
INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1666324800_39149523
INFO:ibmpairs.paw:Here we go, PAIRS query result successfully downloaded as '1666324800_39149523_67ff286cc740d8b40ec9076eac4ec5b0.zip'.
INFO:ibmpairs.paw:Data acknowledgement successfully loaded, print with `self.print_data_acknowledgement()`
INFO:ibmpairs.paw:PAIRS meta data loaded from 'output.info'.


In [15]:
query_vpd.data

{'Expression-temp_rankine[temp_rankine]-Exp': array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]),
 'Expression-temp_celcius[temp_celcius]-Exp': array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]),
 'Expression-partial_pressure[partial_pressure]-Exp': array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]),
 '

In [17]:
dewpoint_celcius = query_vpd.data['Expression-dewpoint_celcius[dewpoint_celcius]-Exp']
dewpoint_celcius[~np.isnan(dewpoint_celcius)]

array([9.68776989, 9.82855034, 9.42827988, 9.60317039])

In [18]:
temp_celcius = query_vpd.data['Expression-temp_celcius[temp_celcius]-Exp']
temp_celcius[~np.isnan(temp_celcius)]

array([10.64000034, 10.90443993, 10.53555012, 10.8597002 ])

In [19]:
relative_humidity = query_vpd.data['Expression-relative_humidity[relative_humidity]-Exp']
relative_humidity[~np.isnan(relative_humidity)]

array([93.83592987, 93.07452393, 92.85902405, 91.95146942])

In [22]:
temp_rankine = query_vpd.data['Expression-temp_rankine[temp_rankine]-Exp']
temp_rankine[~np.isnan(temp_rankine)]

array([510.82199097, 511.29800415, 510.63400269, 511.21746826])

In [20]:
saturation_pressure = query_vpd.data['Expression-saturation[saturation]-Exp']
saturation_pressure[~np.isnan(saturation_pressure)]

array([0.18588842, 0.18919078, 0.18459827, 0.18862846])

In [21]:
vpd = query_vpd.data['Expression-vpd[vpd]-Exp']
vpd[~np.isnan(vpd)]

array([0.01145829, 0.01310236, 0.01318212, 0.01518182])